In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
file = "fullbert_dbpedia2016.txt"
#file = "bert_tiny.txt"
#file = "rdf2vec.txt"
file = "LP_transe_epochs=100embs.txt"

In [3]:
f = open(f"/home/olli/gits/Better_Knowledge_Graph_Embeddings/Evaluation-Framework/{file}",'r')
#f=open(f'./{file}','r')
lines = f.readlines()
f.close()

In [4]:
filterfile =  open("/home/olli/gits/Better_Knowledge_Graph_Embeddings/Evaluation-Framework/relevant_entities_clean.txt",'r')
relevant_entities = set([x.strip() for x in filterfile.readlines()])
f.close()

In [5]:
ents = []
vecs = []

for l in lines:
    l = l.split()
    ents.append(l[0])
    vecs.append(np.array(l[1:]))

In [6]:
len(ents)

4331

In [7]:

embedding_size = len(vecs[0])

In [8]:
unclean_mask = []
for i,(v,e)  in enumerate(zip(vecs,ents)):
    if v.shape != (embedding_size,) or e not in relevant_entities:
        unclean_mask.append(i)    

In [9]:
unclean_mask.reverse() 

In [10]:
for i in unclean_mask:
    vecs.pop(i)
    ents.pop(i)

In [11]:
unclean_mask

[4330]

In [12]:
vecs = np.stack(vecs)
ents = np.array(ents)

In [13]:
len(ents)

4330

In [14]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [15]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)


In [16]:
classes_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbr: <http://dbpedia.org/resource>
PREFIX dbo: <http://dbpedia.org/ontology>
SELECT DISTINCT ?obj WHERE{
<%s>  rdf:type ?obj
FILTER strstarts(str(?obj), str(dbo:))}
"""


In [17]:
classes_queries = [classes_query % c for c in ents]

In [18]:
f'classes_{file}.pkl'

'classes_LP_transe_epochs=100embs.txt.pkl'

In [19]:
import pickle
from pathlib import Path
if not Path(f'classes_{file}.pkl').is_file():
    # save this because api slow!
    classes = []
    for q in tqdm(classes_queries):
        try:
            sparql.setQuery(q)
            classes.append(sparql.query().convert())
        except:
            classes.append(None)

    
    pf = open(f'classes_{file}.pkl','wb')
    pickle.dump(classes,pf)
    pf.close()
else:
    pf = open(f'classes_{file}.pkl','rb')
    classes = pickle.load(pf)
    pf.close()

In [20]:
from collections import Counter

In [21]:
classes_counter = Counter()
for i,c in enumerate(classes):
    if c:
        classes[i] = []
        for d in c['results']['bindings']:
            classes_counter[(d['obj']['value'])]+=1
            classes[i].append(d['obj']['value'])
        
    

In [22]:
class_whitelist = ['http://dbpedia.org/ontology/Film','http://dbpedia.org/ontology/EducationalInstitution',
                  'http://dbpedia.org/ontology/Album','http://dbpedia.org/ontology/Company',
                  'http://dbpedia.org/ontology/City','http://dbpedia.org/ontology/Country',]

In [23]:
class_blacklist = ['http://dbpedia.org/ontology/Work','http://dbpedia.org/ontology/MusicalWork','http://dbpedia.org/ontology/Place','http://dbpedia.org/ontology/PopulatedPlace']

In [24]:
classes_counter

Counter({'http://dbpedia.org/ontology/Work': 1761,
         'http://dbpedia.org/ontology/Organisation': 1557,
         'http://dbpedia.org/ontology/Agent': 1556,
         'http://dbpedia.org/ontology/Album': 1458,
         'http://dbpedia.org/ontology/MusicalWork': 1390,
         'http://dbpedia.org/ontology/Company': 1047,
         'http://dbpedia.org/ontology/University': 860,
         'http://dbpedia.org/ontology/EducationalInstitution': 701,
         'http://dbpedia.org/ontology/PopulatedPlace': 418,
         'http://dbpedia.org/ontology/Place': 417,
         'http://dbpedia.org/ontology/Location': 415,
         'http://dbpedia.org/ontology/Film': 403,
         'http://dbpedia.org/ontology/Settlement': 295,
         'http://dbpedia.org/ontology/City': 248,
         'http://dbpedia.org/ontology/Bank': 147,
         'http://dbpedia.org/ontology/Country': 129,
         'http://dbpedia.org/ontology/Person': 93,
         'http://dbpedia.org/ontology/MusicalArtist': 77,
         'http://

In [25]:
final_classes_big = []
final_classes_small = []

for cs in classes:
    if cs:
        
        biggest_count = -1
        biggest_class = 'http://dbpedia.org/ontology/None'

        for c in cs:
            print(c, c in class_whitelist)
            if c not in class_whitelist:
                continue
            if classes_counter[c] > biggest_count:
                biggest_count = classes_counter[c]
                biggest_class = c
                
        final_classes_big.append(biggest_class)


        smallest_count = -1
        smallest_class = 'http://dbpedia.org/ontology/None'

        for c in cs:
            if c not in class_whitelist:
                continue
            if classes_counter[c] < smallest_count:
                smallest_count = classes_counter[c]
                smallest_class = c
                
        final_classes_small.append(smallest_class)
    else:
        final_classes_big.append('http://dbpedia.org/ontology/None')
        final_classes_small.append('http://dbpedia.org/ontology/None')
        
        

        
        
        
        

http://dbpedia.org/ontology/Work False
http://dbpedia.org/ontology/MusicalWork False
http://dbpedia.org/ontology/Album True
http://dbpedia.org/ontology/Work False
http://dbpedia.org/ontology/MusicalWork False
http://dbpedia.org/ontology/Album True
http://dbpedia.org/ontology/Work False
http://dbpedia.org/ontology/MusicalWork False
http://dbpedia.org/ontology/Album True
http://dbpedia.org/ontology/Work False
http://dbpedia.org/ontology/MusicalWork False
http://dbpedia.org/ontology/Album True
http://dbpedia.org/ontology/Film True
http://dbpedia.org/ontology/Album True
http://dbpedia.org/ontology/Work False
http://dbpedia.org/ontology/MusicalWork False
http://dbpedia.org/ontology/Album True
http://dbpedia.org/ontology/Work False
http://dbpedia.org/ontology/MusicalWork False
http://dbpedia.org/ontology/Album True
http://dbpedia.org/ontology/Work False
http://dbpedia.org/ontology/MusicalWork False
http://dbpedia.org/ontology/Album True
http://dbpedia.org/ontology/Work False
http://dbpedia.o

In [26]:
import sklearn.manifold
reducer = sklearn.manifold.TSNE(learning_rate='auto',init='pca').fit_transform
red = reducer(vecs)

In [27]:
import plotly.express as px

In [28]:
import pandas as pd

In [29]:
len(final_classes_big)

4330

In [30]:
(6605-687)/6605

0.8959878879636639

In [31]:
sum(np.array(final_classes_big) == 'http://dbpedia.org/ontology/None')

350

In [32]:
df = pd.DataFrame(dict(x=red[:,0],y=red[:,1],c = final_classes_big,d=ents))
df = df[df['c'] != 'http://dbpedia.org/ontology/None']

In [33]:
df = df.sort_values(by='c')

In [34]:
from urllib.parse import urlparse

In [35]:
 urlparse('http://dbpedia.org/ontology/Film')

ParseResult(scheme='http', netloc='dbpedia.org', path='/ontology/Film', params='', query='', fragment='')

In [36]:
df['c'] = df['c'].apply(lambda x: x.split('/')[-1])

In [37]:
colors = ['#b35806',
'#f1a340',
'#fee0b6',
'#d8daeb',
'#998ec3',
'#542788']

In [38]:
colors = ['#8dd3c7',
 '#e6e600',
'#bebada',
'#fb8072',
'#80b1d3',
'#fdb462']

In [39]:
colors = ['#e41a1c',
'#377eb8',
'#4daf4a',
'#984ea3',
'#ff7f00',
'#e6e600',]

In [46]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(df,x='x',y='y',color='c',hover_name='d',color_discrete_sequence=colors,labels={
                     "x":"",# "<b>RDF2Vec</b> t-SNE feature 0",#"<b>BERT<sub>TINY</sub> </b>t-SNE feature 0",
                     "y":"",# "<b>RDF2Vec</b> t-SNE feature 1",#"<b>BERT<sub>TINY</sub> </b> t-SNE feature 1""<b>RDF2Vec</b> t-SNE feature 1",
                    "c":"<b>rdf:type label</b>",
                 })#,title="bertTiny Datamining tsne embeddings")
fig.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
            ,showlegend=True,
                  paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    legend = {'font': {'size':13},'itemsizing': 'constant', 'entrywidth':0.3, # change it to 0.3
                               'entrywidthmode':'fraction',
                              'orientation':'h',
                              'y':1.2,
                              'xanchor':"center",
                              'x':0.5}
                 )



fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
  )
fig.write_image(f'{file}_classes.pdf')

In [41]:
print(f'{file}_classes.pdf')

LP_transe_epochs=100embs.txt_classes.pdf
